In [1]:
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from keras.layers import Embedding
import pandas as pd 
import numpy as np
import json

In [2]:
#import training data 
f = open('training_features_binary_allCaps.json')
feats = json.load(f)

In [3]:
#pulling expected labels from JSON
expected_labels = []
for x in feats: 
    expected_labels.append(feats.get(x)['label'])

expected_labels = np.asarray(expected_labels)
np.shape(expected_labels)


(1007,)

In [4]:
#pulling input_ids 
word_index_ids = []
for x in feats: 
    word_index_ids.append(feats.get(x)['indexes'])

#padding input_ids 
max_sequence_length = 512

padded_word_index_ids = []
for input_id in word_index_ids:
    if len(input_id) < max_sequence_length:
      padded_word_index_ids.append(input_id + [0] * (max_sequence_length - len(input_id)))
    else:
      padded_word_index_ids.append(input_id)

#convert to array
padded_word_index_ids = np.asarray(padded_word_index_ids)

#final shape
np.shape(padded_word_index_ids)

(1007, 512)

In [5]:
#pulling caption_ids 
cap_index_ids = []
for x in feats: 
    cap_index_ids.append(feats.get(x)['caps_index'])

#padding caps_index
max_sequence_length = 512

padded_cap_index_ids = []
for input_id in cap_index_ids:
    if len(input_id) < max_sequence_length:
      padded_cap_index_ids.append(input_id + [0] * (max_sequence_length - len(input_id)))
    else:
      padded_cap_index_ids.append(input_id)


#convert to array
padded_cap_index_ids = np.asarray(padded_cap_index_ids)

#final shape
np.shape(padded_cap_index_ids)


(1007, 512)

In [21]:
def model(i,input_1,input_2,labels): 
    #concat. inputs 
    x = keras.Input(i,)
    y = keras.Input(i,)
    conc = layers.Concatenate(axis=-1)([x,y])
    
    # Dense layers 
    layer = layers.Dense(512, activation='sigmoid')(conc)
    layer = layers.Dense(512, activation='sigmoid')(layer)
    layer = layers.Dense(512, activation='sigmoid')(layer)
    out = layers.Dense(1, activation='sigmoid')(layer)

    #Compling model
    model = keras.Model(inputs=[x, y], outputs=out)
    
    sgd = SGD(learning_rate=0.1,momentum=0.25, nesterov=True)
    model.compile(optimizer = sgd, loss='binary_crossentropy', metrics=['accuracy'])
    
    n = model.fit([input_1, input_2],labels, batch_size=30, epochs=500)
    model.summary()
    

    return n


In [22]:
model(512, padded_word_index_ids, padded_cap_index_ids, expected_labels)

Epoch 1/500
34/34 [==============================] - 3s 24ms/step - loss: 1.1564 - accuracy: 0.5809
Epoch 2/500
34/34 [==============================] - 1s 23ms/step - loss: 0.7598 - accuracy: 0.6276
Epoch 3/500
34/34 [==============================] - 1s 19ms/step - loss: 0.6852 - accuracy: 0.6425
Epoch 4/500
34/34 [==============================] - 1s 20ms/step - loss: 0.6238 - accuracy: 0.6931
Epoch 5/500
34/34 [==============================] - 1s 19ms/step - loss: 0.6497 - accuracy: 0.6653
Epoch 6/500
34/34 [==============================] - 1s 20ms/step - loss: 0.6582 - accuracy: 0.6653
Epoch 7/500
34/34 [==============================] - 1s 22ms/step - loss: 0.6380 - accuracy: 0.6584
Epoch 8/500
34/34 [==============================] - 1s 18ms/step - loss: 0.6290 - accuracy: 0.6832
Epoch 9/500
34/34 [==============================] - 1s 22ms/step - loss: 0.6225 - accuracy: 0.6912
Epoch 10/500
34/34 [==============================] - 1s 24ms/step - loss: 0.6342 - accuracy: 0.6802